In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from datetime import datetime
import openai
import os


os.environ["OPENAI_API_KEY"] = ""

PDF parser

In [ ]:
def produce_question(resume):
    loader = PyMuPDFLoader(resume)
    data = loader.load()
    text = ''
    for page in data:
        text += page.page_content
    return text

Sample resume text

In [ ]:
text = '''Mandy Campbell, Software Engineer
PROFILE
Passionate, fresh software engineer graduate from UCLA. Has competed in multiple team-based coding competitions.
EMPLOYMENT HISTORY
Jan 2021 — Aug 2021
FRONTEND DEVELOPER INTERN, XYZ Enterprise
Mostly did debugging on the new features developed by the supervisors. Participated in daily scrum meetings and assisted in making decisions on UI.
• Learnt about technology currently used in the industry
• Able to have hands experience with creating/modifying UI
EDUCATION
Jul 2021 — Present
Bachelor's Degree in Software Engineering, UCLA
• Student representative for the software engineering faculty.
• Leader of the competitive coding society
Jan 2020 — Jul 2021
Advanced Computer Science Diploma, UCLA
Jan 2018 — Dec 2018
High School Diploma, Barlow High School
Tennessee
• Leader of tech club
• Treasurer for student council
SKILLS
Python
Javascript
Mobile development
Artificial Intelligence
HOBBIES
Skiing, Hockey, Knitting
LANGUAGES
English
Native speaker
ACCOMPLISHMENTS
• Longest running student representative
• 1st place winner of the national coding competition 2022
• Teaching assistant for advanced algorithms class
'''


Prompt used


In [2]:
'''
- Ask 3 questions
- Questions would be related to the resume + job/company description
- Once the conversation has ended, let backend know by responding with '_'
- Engage with candidate like a conversation as opposed to asking all the questions at once
'''
prompt = 'You will ask me only 3 questions about my resume, one question at a time to me. You do not answer as a candidate. \
    I will first provide you a resume, then you ask the question. Once the interview is completed, respond with an underscore'

You will ask me only 3 questions about my resume, one question at a time to me. You do not answer as a candidate.     I will first provide you a resume, then you ask the question. Once the interview is completed, respond with an underscore


First approach: Call the function everytime a new conversation is created

In [ ]:
def start_conversation(company_name, company_desc, job_title, resume_text):
    #Provide information on the company and what is the job vacancy
    company_background = f'Company name:{company_name}, company description:{company_desc}, job title:{job_title}'
    # Setting up the initial chat/prompt for gpt
    chat_session = [{'role': 'system', 'content': f'Company background: {company_background}'},
                    {'role': 'system', 'content': prompt},
                    {'role': 'user', 'content': resume_text}]
    #Logs will contain the conversation between the user and gpt, storing the timestamp of the message as well
    logs = []
    # Loop until end of conversation
    while not '_' in chat_session[-1]['content']:
        # Retrieving response based on user input
        response = openai.ChatCompletion.create(
            model='gpt-4', messages=chat_session)
        message = response['choices'][0]['message']['content']
        # Check for _ (From the prompt, gpt will respond with '_' if conversation is completed)
        if '_' in message:
            break
        else:
            print(message)
        
        #Add the response into the chat session and the logs
        chat_session.append({'role': 'assistant', 'content': message})
        logs.append({'role': 'assistant', 'content': message,
                    'time': str(datetime.now())})
        
        #Get user input
        user_input = ''
        user_input = input()
        print(user_input)
        
        #Add user input to chat session and logs
        chat_session.append({'role': 'user', 'content': user_input})
        logs.append({'role': 'user', 'content': user_input,
                    'time': str(datetime.now())})
        
    #Return the logs to be saved in database
    return logs

Second approach: Have backend call functions multiple times per conversation

In [ ]:
#Will be called only once per conversation at the start
def start_conversation(company_name, company_desc, job_title, resume_text):
    #Provide information on the company and what is the job vacancy
    company_background = f'Company name:{company_name}, company description:{company_desc}, job title:{job_title}'
    # Setting up the initial chat/prompt for gpt
    chat_session = [{'role': 'system', 'content': f'Company background: {company_background}'},
                    {'role': 'system', 'content': prompt},
                    {'role': 'user', 'content': resume_text}]
    #Generate the first question to candidate
    response = openai.ChatCompletion.create(
        model='gpt-4', messages=chat_session)
    response_msg = response['choices'][0]['message']['content']
    #Return the message to backend to store and be displayed
    return response_msg

#Will be called multiple times in the conversation
def conversation(chat_session, user_input):
    #Add user input to the chat session and generate a response
    message = {'role': 'user', 'content': user_input}
    chat_session.append(message)
    response = openai.ChatCompletion.create(
        model='gpt-4', messages=chat_session)
    response_msg = response['choices'][0]['message']['content']
    #Modify the message to include a timestamp
    message.update({'time': str(datetime.now)})
    #Return the response and the data to be stored in database
    data = {'response_msg': response_msg, 'log_msg': message}
    return data
